In [0]:
%python

jdbc_hostname = dbutils.secrets.get(scope='dbx-scope', key='oracle-hostname')      # e.g. "db.myvps.example.com"
jdbc_port     = "1521"
service_name  = dbutils.secrets.get(scope='dbx-scope', key='oracle-service-name')  # e.g. "SAMPLEFREEPDB1"
jdbc_user     = dbutils.secrets.get(scope='dbx-scope', key='oracle-user')
jdbc_password = dbutils.secrets.get(scope='dbx-scope', key='oracle-password')

jdbc_url = f"jdbc:oracle:thin:@//{jdbc_hostname}:{jdbc_port}/{service_name}"
driver   = "oracle.jdbc.OracleDriver"   # from ojdbc17.jar installed in the server

In [0]:
test_df = (spark.read.format("jdbc")
  .option("url", jdbc_url)
  .option("dbtable", "(SELECT * FROM ORDERS WHERE ROWNUM <= 10) t")
  .option("user", jdbc_user)
  .option("password", jdbc_password)
  .option("driver", driver)
  .option("fetchsize", "10")
  .load())

display(test_df)   # or: test_df.show()

## CDC from Oracle

In [0]:
%python

jdbc_url    = f"jdbc:oracle:thin:@//{jdbc_hostname}:{jdbc_port}/{service_name}"
jdbc_driver = "oracle.jdbc.OracleDriver"

source_table = "ORDERS"            # add schema if needed, e.g. MYSCHEMA.ORDERS
pk_col       = "ORDER_ID"
ts_col       = "ORDER_TIMESTAMP"

target_table    = "delta_db.orders_delta"
watermark_table = "delta_db._wm_orders"

batch_limit_rows       = 10000
safety_lag_seconds     = 30
max_retries            = 5

jdbc_read_options = {"fetchsize": "100"}

In [0]:
%sql
USE DATABASE DELTA_DB;

In [0]:
# Create the target with the real schema
spark.sql("""
CREATE TABLE IF NOT EXISTS delta_db.orders_delta (
  ORDER_ID        BIGINT,
  ORDER_UUID      STRING,
  TEMPERATURE     DOUBLE,
  ORDER_TIMESTAMP TIMESTAMP
)
USING DELTA
""")

# Create WM table if missing
spark.sql(f"""
CREATE TABLE IF NOT EXISTS delta_db._wm_orders (
  table_name STRING,
  last_ts    TIMESTAMP,
  last_id    BIGINT
)
USING DELTA
""")

# Initialize watermark if empty
if spark.table("delta_db._wm_orders").where("table_name = 'ORDERS'").count() == 0:
    spark.sql("""
      INSERT INTO delta_db._wm_orders VALUES ('ORDERS', TIMESTAMP '1970-01-01 00:00:00', 0)
    """)

In [0]:
%python
display(spark.table('_wm_orders').limit(20))
display(spark.table('orders_delta').limit(20))

In [0]:
from datetime import datetime, timedelta, timezone
from pyspark.sql import functions as F

def get_watermark():
    row = (spark.table(watermark_table)
           .where("table_name = 'ORDERS'")
           .select("last_ts", "last_id")
           .first())
    return row["last_ts"], int(row["last_id"])

def set_watermark(new_ts, new_id):
    spark.sql(f"""
      MERGE INTO {watermark_table} AS t
      USING (SELECT 'ORDERS' AS table_name,
                    TIMESTAMP '{new_ts.strftime("%Y-%m-%d %H:%M:%S")}' AS last_ts,
                    {new_id} AS last_id) AS s
      ON t.table_name = s.table_name
      WHEN MATCHED THEN UPDATE SET last_ts = s.last_ts, last_id = s.last_id
      WHEN NOT MATCHED THEN INSERT (table_name, last_ts, last_id)
           VALUES (s.table_name, s.last_ts, s.last_id)
    """)

def read_incremental_batch(last_ts, last_id, limit_rows, safety_lag_s):
    upper_ts   = (datetime.now(timezone.utc) - timedelta(seconds=safety_lag_s)).strftime("%Y-%m-%d %H:%M:%S")
    last_ts_str= last_ts.strftime("%Y-%m-%d %H:%M:%S")

    # Cast ORDER_ID to integer width so it matches BIGINT in the target/watermark
    query = f"""
    SELECT
      CAST({pk_col} AS NUMBER(38,0)) AS {pk_col},
      ORDER_UUID,
      TEMPERATURE,
      {ts_col}
    FROM {source_table}
    WHERE
      ( {ts_col} > TO_TIMESTAMP('{last_ts_str}', 'YYYY-MM-DD HH24:MI:SS')
        OR ( {ts_col} = TO_TIMESTAMP('{last_ts_str}', 'YYYY-MM-DD HH24:MI:SS')
             AND CAST({pk_col} AS NUMBER(38,0)) > {last_id} )
      )
      AND {ts_col} <= TO_TIMESTAMP('{upper_ts}', 'YYYY-MM-DD HH24:MI:SS')
    ORDER BY {ts_col}, CAST({pk_col} AS NUMBER(38,0))
    FETCH FIRST {limit_rows} ROWS ONLY
    """

    reader = (spark.read.format("jdbc")
              .option("url", jdbc_url)
              .option("dbtable", f"({query}) t")
              .option("user", jdbc_user)
              .option("password", jdbc_password)
              .option("driver", jdbc_driver))
    for k,v in jdbc_read_options.items():
        reader = reader.option(k,v)
    return reader.load()

def cdc_once():
    last_ts, last_id = get_watermark()
    df = read_incremental_batch(last_ts, last_id, batch_limit_rows, safety_lag_seconds)

    if df.head(1) == []:
        return 0, last_ts, last_id

    # Idempotent upsert. Columns on both sides match the target we created above.
    df.createOrReplaceTempView("stg_orders")
    spark.sql(f"""
      MERGE INTO {target_table} AS tgt
      USING stg_orders AS src
      ON tgt.{pk_col} = src.{pk_col}
      WHEN MATCHED THEN UPDATE SET
        tgt.ORDER_ID        = src.ORDER_ID,
        tgt.ORDER_UUID      = src.ORDER_UUID,
        tgt.TEMPERATURE     = src.TEMPERATURE,
        tgt.ORDER_TIMESTAMP = src.ORDER_TIMESTAMP
      WHEN NOT MATCHED THEN INSERT (
        ORDER_ID, ORDER_UUID, TEMPERATURE, ORDER_TIMESTAMP
      ) VALUES (
        src.ORDER_ID, src.ORDER_UUID, src.TEMPERATURE, src.ORDER_TIMESTAMP
      )
    """)

    max_ts = df.agg(F.max(F.col(ts_col)).alias("m")).collect()[0]["m"]
    max_id = (df.filter(F.col(ts_col) == F.lit(max_ts))
                .agg(F.max(F.col(pk_col)).alias("id")).collect()[0]["id"])

    set_watermark(max_ts, int(max_id))
    return df.count(), max_ts, int(max_id)

## Testing

In [0]:
rows, new_ts, new_id = cdc_once()
print(f"[TEST] Upserted {rows} rows; watermark → {new_ts} / {new_id}")

display(spark.table(target_table).limit(20))
display(spark.table(watermark_table))

# Oracle → Databricks CDC Approaches

## Option 1: Polling as a Job (Scheduled|Continuous)

In this approach, the CDC process is executed once per run and scheduled using Databricks Jobs.  
The job runs on a defined interval (for example, every minute), queries Oracle for new changes since the last watermark, and updates the Delta table.  
Each execution starts, processes the incremental data, updates the watermark, and then stops.

**Characteristics:**
- Latency depends on the schedule (e.g., 1–5 minutes).  
- Cost-efficient: clusters can terminate between runs.  
- Easy to monitor and retry using the Databricks Jobs interface.  
- Good for demos and PoCs where near-real-time (not sub-second) is acceptable.  

---

## Option 2: Infinite Loop (Always-On)

In this approach, the CDC process runs continuously inside a notebook or job.  
A `while` loop keeps polling Oracle on a short interval, applying changes to the Delta table and updating the watermark without stopping.  
The notebook remains active and constantly looks for new changes.

**Characteristics:**
- Lower latency, as changes are picked up almost immediately.  
- Requires a long-running cluster, which increases cost.  
- Needs timeout/restart configuration to ensure resiliency.  
- Useful for demos where continuous updates must be visible without waiting for the next job run.  

---

In [0]:
# Hard reset
#spark.sql("DROP TABLE IF EXISTS delta_db.orders_delta")
#spark.sql("DROP TABLE IF EXISTS delta_db._wm_orders")